# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 24, 10, 25

while True:
    dt = datetime.now()
    if dt.month >= month and dt.day >= day:
        if dt.hour > hour or (dt.hour == hour and dt.minute > minute):
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 300
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [15, 20, 25, 30, 35]},
              'STOCH': {'fastk_period': [5, 7, 9, 11], 'slowk_period': [2, 3, 4, 5],
                        'slowd_period': [3, 5, 7, 9], 'low_bound': [10, 15, 20, 25]}}
# optim_dict = {'RSI': {'timeperiod': [16], 'low_bound': [30]},
#                   'STOCH': {'fastk_period': [5], 'slowk_period': [4],
#                             'slowd_period': [2], 'low_bound': [30]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['indicator_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat = stat.sort_values(['indicator_rank', 'pct_right_forecast_avg'], ascending=False)
stat.head(20)

Number of combinations is 3840


100%|████████████████████████████████████████████████████████████████| 3840/3840 [1:47:33<00:00,  1.68s/it]


,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,indicator_rank
2595,STOCH_RSI,16,15,5,4,3,25,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,-1.84,-1.90,-2.41,-1.86,-1.96,-2.07,-2.08,-2.24,-2.40,-2.83,-2.74,-2.78,-2.93,-2.99,-3.02,-3.09,-3.06,-3.14,-3.18,-3.47,-3.57,-3.67,-3.68,-4.44,6,100.000000,180.000000
2658,STOCH_RSI,16,15,7,4,3,20,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,-1.84,-1.90,-2.41,-1.86,-1.96,-2.08,-2.08,-2.24,-2.27,-2.83,-2.74,-2.78,-2.93,-3.00,-3.02,-3.39,-3.44,-3.46,-3.62,-4.19,-4.22,-3.92,-3.68,-4.00,6,100.000000,180.000000
2659,STOCH_RSI,16,15,7,4,3,25,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,-1.84,-1.90,-2.41,-1.86,-1.96,-2.08,-2.08,-2.24,-2.27,-2.83,-2.74,-2.78,-2.93,-3.00,-3.02,-3.39,-3.44,-3.46,-3.62,-4.19,-4.22,-3.92,-3.68,-4.00,6,100.000000,180.000000
1367,STOCH_RSI,14,15,7,3,5,25,100.00,83.33,83.33,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,-1.56,-1.90,-2.41,-1.86,-1.48,-1.58,-1.40,-1.22,-1.52,-1.64,-1.70,-1.71,-1.76,-1.87,-2.32,-2.25,-2.23,-2.38,-2.72,-2.83,-3.16,-3.08,-3.23,-3.00,6,98.610833,171.665000
2647,STOCH_RSI,16,15,7,3,5,25,100.00,83.33,83.33,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,-1.56,-1.90,-2.41,-1.86,-1.48,-1.58,-1.40,-1.22,-1.52,-1.64,-1.70,-1.71,-1.76,-1.87,-2.32,-2.25,-2.23,-2.38,-2.72,-2.83,-3.16,-3.08,-3.23,-3.00,6,98.610833,171.665000
1763,STOCH_RSI,14,20,11,4,3,25,86.67,73.33,73.33,73.33,73.33,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,86.67,86.67,86.67,86.67,86.67,86.67,86.67,-0.85,-0.85,-1.34,-1.36,-1.79,-2.18,-2.24,-2.16,-1.73,-1.62,-1.81,-2.40,-2.55,-2.69,-2.50,-2.40,-2.43,-2.69,-2.75,-2.62,-2.47,-2.58,-2.50,-2.47,15,81.111667,166.675000
1699,STOCH_RSI,14,20,9,4,3,25,92.86,78.57,78.57,78.57,78.57,78.57,78.57,78.57,78.57,78.57,78.57,78.57,78.57,78.57,78.57,78.57,78.57,85.71,85.71,85.71,85.71,85.71,85.71,85.71,-0.95,-0.83,-1.19,-2.21,-2.36,-2.36,-2.38,-2.16,-1.94,-1.89,-2.15,-2.52,-2.56,-2.73,-2.73,-2.42,-2.39,-2.61,-2.76,-2.70,-2.59,-2.59,-2.63,-2.78,14,81.247917,157.470833
2627,STOCH_RSI,16,15,7,2,3,25,100.00,100.00,83.33,66.67,83.33,83.33,83.33,83.33,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,-1.51,-1.80,-2.19,-1.80,-1.96,-2.01,-1.80,-1.62,-1.90,-1.91,-2.31,-2.41,-2.44,-2.54,-2.71,-2.85,-2.53,-2.23,-2.23,-2.29,-2.57,-2.79,-2.86,-2.56,6,95.138333,150.830000
2691,STOCH_RSI,16,15,9,2,3,

# Save new config data to config file

In [3]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'RSI': {'timeperiod': [16], 'low_bound': [15]},
                  'STOCH': {'fastk_period': [5], 'slowk_period': [4],
                            'slowd_period': [3], 'low_bound': [25]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)